Disabled automatic broadcast join with spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import  broadcast, expr, col
spark = SparkSession.builder \
                    .appName("Jupyter") \
                    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
                    .getOrCreate()


24/12/17 19:31:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
df_match_details = spark.read.option("header", "true").csv("/home/iceberg/data/match_details.csv")
df_match_details.head()

Row(match_id='71d79b23-4143-4359-a62e-489a27597b23', player_gamertag='taterbase', previous_spartan_rank='5', spartan_rank='5', previous_total_xp='12537', total_xp='13383', previous_csr_tier='1', previous_csr_designation='3', previous_csr='0', previous_csr_percent_to_next_tier='98', previous_csr_rank=None, current_csr_tier='2', current_csr_designation='3', current_csr='0', current_csr_percent_to_next_tier='26', current_csr_rank=None, player_rank_on_team='4', player_finished='false', player_average_life='PT14.81149S', player_total_kills='6', player_total_headshots='4', player_total_weapon_damage='255', player_total_shots_landed='28', player_total_melee_kills='0', player_total_melee_damage='0', player_total_assassinations='0', player_total_ground_pound_kills='0', player_total_shoulder_bash_kills='0', player_total_grenade_damage='0', player_total_power_weapon_damage='0', player_total_power_weapon_grabs='0', player_total_deaths='13', player_total_assists='1', player_total_grenade_kills='0',

In [6]:
df_matches = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")
df_matches.head()

Row(match_id='11de1a94-8d07-4162-9f5f-d3cc753c811c', mapid='c7edbf0f-f206-11e4-aa52-24be05e24f7e', is_team_game='true', playlist_id='f72e0ef0-7c4a-4307-af78-8e38dac3fdba', game_variant_id='1e473914-46e4-408d-af26-178fb115de76', is_match_over='true', completion_date='2016-02-22 00:00:00.000000', match_duration=None, game_mode=None, map_variant_id=None)

In [7]:
df_medals_matches_players = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")
df_medals_matches_players.head()

Row(match_id='009fdac5-e15c-47c6-a202-e18ff8800ce7', player_gamertag='EcZachly', medal_id='3261908037', count='7')

In [8]:
df_medals = spark.read.option("header", "true").csv("/home/iceberg/data/medals.csv")
df_medals.head()

Row(medal_id='2315448068', sprite_uri=None, sprite_left=None, sprite_top=None, sprite_sheet_width=None, sprite_sheet_height=None, sprite_width=None, sprite_height=None, classification=None, description=None, name=None, difficulty=None)

In [9]:
df_maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")
df_maps.head()

Row(mapid='c93d708f-f206-11e4-a815-24be05e24f7e', name='Urban', description='Andesia was the crucible for countless heroes and villains caught in the throes of seething rebellion and righteous excess.')

Bucket join match_details, matches, and medal_matches_players on match_id with 16 buckets

In [16]:
for i in df_match_details.dtypes:
    print(f"{i[0]} {i[1]} ,")

match_id string ,
player_gamertag string ,
previous_spartan_rank string ,
spartan_rank string ,
previous_total_xp string ,
total_xp string ,
previous_csr_tier string ,
previous_csr_designation string ,
previous_csr string ,
previous_csr_percent_to_next_tier string ,
previous_csr_rank string ,
current_csr_tier string ,
current_csr_designation string ,
current_csr string ,
current_csr_percent_to_next_tier string ,
current_csr_rank string ,
player_rank_on_team string ,
player_finished string ,
player_average_life string ,
player_total_kills string ,
player_total_headshots string ,
player_total_weapon_damage string ,
player_total_shots_landed string ,
player_total_melee_kills string ,
player_total_melee_damage string ,
player_total_assassinations string ,
player_total_ground_pound_kills string ,
player_total_shoulder_bash_kills string ,
player_total_grenade_damage string ,
player_total_power_weapon_damage string ,
player_total_power_weapon_grabs string ,
player_total_deaths string ,
player

In [18]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
    match_id string ,
    player_gamertag string ,
    previous_spartan_rank string ,
    spartan_rank string ,
    previous_total_xp string ,
    total_xp string ,
    previous_csr_tier string ,
    previous_csr_designation string ,
    previous_csr string ,
    previous_csr_percent_to_next_tier string ,
    previous_csr_rank string ,
    current_csr_tier string ,
    current_csr_designation string ,
    current_csr string ,
    current_csr_percent_to_next_tier string ,
    current_csr_rank string ,
    player_rank_on_team string ,
    player_finished string ,
    player_average_life string ,
    player_total_kills string ,
    player_total_headshots string ,
    player_total_weapon_damage string ,
    player_total_shots_landed string ,
    player_total_melee_kills string ,
    player_total_melee_damage string ,
    player_total_assassinations string ,
    player_total_ground_pound_kills string ,
    player_total_shoulder_bash_kills string ,
    player_total_grenade_damage string ,
    player_total_power_weapon_damage string ,
    player_total_power_weapon_grabs string ,
    player_total_deaths string ,
    player_total_assists string ,
    player_total_grenade_kills string ,
    did_win string ,
    team_id string 
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));

++
||
++
++

In [19]:
df_match_details.write.mode("append").bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")

In [20]:
for i in df_matches.dtypes:
    print(f"{i[0]} {i[1]} ,")

match_id string ,
mapid string ,
is_team_game string ,
playlist_id string ,
game_variant_id string ,
is_match_over string ,
completion_date string ,
match_duration string ,
game_mode string ,
map_variant_id string ,


In [21]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id string ,
    mapid string ,
    is_team_game string ,
    playlist_id string ,
    game_variant_id string ,
    is_match_over string ,
    completion_date string ,
    match_duration string ,
    game_mode string ,
    map_variant_id string 
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));

++
||
++
++

In [22]:
df_matches.write.mode("append").bucketBy(16, "match_id").saveAsTable("bootcamp.matches_bucketed")

In [23]:
for i in df_medals_matches_players.dtypes:
    print(f"{i[0]} {i[1]} ,")

match_id string ,
player_gamertag string ,
medal_id string ,
count string ,


In [24]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.medals_matches_players_bucketed (
    match_id string ,
    player_gamertag string ,
    medal_id string ,
    count string
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));

++
||
++
++

In [25]:
df_medals_matches_players.write \
                        .mode("append") \
                        .bucketBy(16, "match_id") \
                        .saveAsTable("bootcamp.medals_matches_players_bucketed")

In [54]:
%%sql

    -- select count(*)
    -- from(
SELECT a.* 
    ,b.mapid
    ,b.is_team_game
    ,b.playlist_id
    ,b.game_variant_id
    ,b.is_match_over
    ,b.completion_date
    ,b.match_duration
    ,b.game_mode
    ,b.map_variant_id
    ,c.player_gamertag as player_gamertag2
    ,c.medal_id
    ,c.`count`
FROM bootcamp.match_details_bucketed as a 
JOIN bootcamp.matches_bucketed as b 
ON a.match_id = b.match_id
JOIN bootcamp.medals_matches_players_bucketed as c 
ON a.match_id = c.match_id and  a.player_gamertag = c.player_gamertag
    -- ) as a

match_id,player_gamertag,previous_spartan_rank,spartan_rank,previous_total_xp,total_xp,previous_csr_tier,previous_csr_designation,previous_csr,previous_csr_percent_to_next_tier,previous_csr_rank,current_csr_tier,current_csr_designation,current_csr,current_csr_percent_to_next_tier,current_csr_rank,player_rank_on_team,player_finished,player_average_life,player_total_kills,player_total_headshots,player_total_weapon_damage,player_total_shots_landed,player_total_melee_kills,player_total_melee_damage,player_total_assassinations,player_total_ground_pound_kills,player_total_shoulder_bash_kills,player_total_grenade_damage,player_total_power_weapon_damage,player_total_power_weapon_grabs,player_total_deaths,player_total_assists,player_total_grenade_kills,did_win,team_id,mapid,is_team_game,playlist_id,game_variant_id,is_match_over,completion_date,match_duration,game_mode,map_variant_id,player_gamertag2,medal_id,count
0001a1c4-83dc-4f40-a97e-7910a765c96a,ILLICIT 117,69,69,1658879,1660583,None,None,None,None,None,None,None,None,None,None,9,false,PT23.6586654S,23,8,2844.956983089447,234,1,115,0,0,2,283.0304431915283,968.3815231323242,0,28,10,0,0,1,c7805740-f206-11e4-982c-24be05e24f7e,true,780cc101-005c-4fca-8ce7-6f36d7156ffe,b45854a7-e6e1-4a9c-9104-139934511779,true,2016-01-06 00:00:00.000000,None,None,51db9164-a000-4623-a591-e886c2705dd8,ILLICIT 117,3565443938,4
0001a1c4-83dc-4f40-a97e-7910a765c96a,ILLICIT 117,69,69,1658879,1660583,None,None,None,None,None,None,None,None,None,None,9,false,PT23.6586654S,23,8,2844.956983089447,234,1,115,0,0,2,283.0304431915283,968.3815231323242,0,28,10,0,0,1,c7805740-f206-11e4-982c-24be05e24f7e,true,780cc101-005c-4fca-8ce7-6f36d7156ffe,b45854a7-e6e1-4a9c-9104-139934511779,true,2016-01-06 00:00:00.000000,None,None,51db9164-a000-4623-a591-e886c2705dd8,ILLICIT 117,3261908037,8
0001a1c4-83dc-4f40-a97e-7910a765c96a,ILLICIT 117,69,69,1658879,1660583,None,None,None,None,None,None,None,None,None,None,9,false,PT23.6586654S,23,8,2844.956983089447,234,1,115,0,0,2,283.0304431915283,968.3815231323242,0,28,10,0,0,1,c7805740-f206-11e4-982c-24be05e24f7e,true,780cc101-005c-4fca-8ce7-6f36d7156ffe,b45854a7-e6e1-4a9c-9104-139934511779,true,2016-01-06 00:00:00.000000,None,None,51db9164-a000-4623-a591-e886c2705dd8,ILLICIT 117,824733727,1
0001a1c4-83dc-4f40-a97e-7910a765c96a,ILLICIT 117,69,69,1658879,1660583,None,None,None,None,None,None,None,None,None,None,9,false,PT23.6586654S,23,8,2844.956983089447,234,1,115,0,0,2,283.0304431915283,968.3815231323242,0,28,10,0,0,1,c7805740-f206-11e4-982c-24be05e24f7e,true,780cc101-005c-4fca-8ce7-6f36d7156ffe,b45854a7-e6e1-4a9c-9104-139934511779,true,2016-01-06 00:00:00.000000,None,None,51db9164-a000-4623-a591-e886c2705dd8,ILLICIT 117,298813630,2
0001a1c4-83dc-4f40-a97e-7910a765c96a,ILLICIT 117,69,69,1658879,1660583,None,None,None,None,None,None,None,None,None,None,9,false,PT23.6586654S,23,8,2844.956983089447,234,1,115,0,0,2,283.0304431915283,968.3815231323242,0,28,10,0,0,1,c7805740-f206-11e4-982c-24be05e24f7e,true,780cc101-005c-4fca-8ce7-6f36d7156ffe,b45854a7-e6e1-4a9c-9104-139934511779,true,2016-01-06 00:00:00.000000,None,None,51db9164-a000-4623-a591-e886c2705dd8,ILLICIT 117,1351381581,3
0001a1c4-83dc-4f40-a97e-7910a765c96a,ILLICIT 117,69,69,1658879,1660583,None,None,None,None,None,None,None,None,None,None,9,false,PT23.6586654S,23,8,2844.956983089447,234,1,115,0,0,2,283.0304431915283,968.3815231323242,0,28,10,0,0,1,c7805740-f206-11e4-982c-24be05e24f7e,true,780cc101-005c-4fca-8ce7-6f36d7156ffe,b45854a7-e6e1-4a9c-9104-139934511779,true,2016-01-06 00:00:00.000000,None,None,51db9164-a000-4623-a591-e886c2705dd8,ILLICIT 117,2916014239,5
0001a1c4-83dc-4f40-a97e-7910a765c96a,ILLICIT 117,69,69,1658879,1660583,None,None,None,None,None,None,None,None,None,None,9,false,PT23.6586654S,23,8,2844.956983089447,234,1,115,0,0,2,283.0304431915283,968.3815231323242,0,28,10,0,0,1,c7805740-f206-11e4-982c-24be05e24f7e,true,780cc101-005c-4fca-8ce7-6f36d7156ffe,b45854a7-e6e1-4a9c-9104-139934511779,true,2016-01-06 00:00:0

In [86]:
df_joined = spark.sql( 
    """SELECT a.* 
    ,b.mapid
    ,b.is_team_game
    ,b.playlist_id
    ,b.game_variant_id
    ,b.is_match_over
    ,b.completion_date
    ,b.match_duration
    ,b.game_mode
    ,b.map_variant_id
    ,c.medal_id
    ,c.`count`
FROM bootcamp.match_details_bucketed as a 
JOIN bootcamp.matches_bucketed as b 
ON a.match_id = b.match_id
JOIN bootcamp.medals_matches_players_bucketed as c 
ON a.match_id = c.match_id and  a.player_gamertag = c.player_gamertag""" 
) 
df_joined.show()

+--------------------+---------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+----------+-----+
|            match_id|player_gamertag|prev

Explicitly broadcast JOINs medals and maps

In [87]:
df_joined = df_joined.join(
  broadcast(df_medals),
  df_medals.medal_id == df_joined.medal_id
).drop(df_medals.medal_id).withColumnRenamed("name", "medal_name").withColumnRenamed("description", "medal_description")

In [88]:
df_joined = df_joined.join(
  broadcast(df_maps),
  df_maps.mapid == df_joined.mapid
).drop(df_maps.mapid).withColumnRenamed("name", "map_name").withColumnRenamed("description", "map_description")

In [89]:
sorted(df_joined.columns)


['classification',
 'completion_date',
 'count',
 'current_csr',
 'current_csr_designation',
 'current_csr_percent_to_next_tier',
 'current_csr_rank',
 'current_csr_tier',
 'did_win',
 'difficulty',
 'game_mode',
 'game_variant_id',
 'is_match_over',
 'is_team_game',
 'map_description',
 'map_name',
 'map_variant_id',
 'mapid',
 'match_duration',
 'match_id',
 'medal_description',
 'medal_id',
 'medal_name',
 'player_average_life',
 'player_finished',
 'player_gamertag',
 'player_rank_on_team',
 'player_total_assassinations',
 'player_total_assists',
 'player_total_deaths',
 'player_total_grenade_damage',
 'player_total_grenade_kills',
 'player_total_ground_pound_kills',
 'player_total_headshots',
 'player_total_kills',
 'player_total_melee_damage',
 'player_total_melee_kills',
 'player_total_power_weapon_damage',
 'player_total_power_weapon_grabs',
 'player_total_shots_landed',
 'player_total_shoulder_bash_kills',
 'player_total_weapon_damage',
 'playlist_id',
 'previous_csr',
 'previ

In [90]:
df_joined.createOrReplaceTempView("df_joined")

Aggregate the joined data frame to figure out questions like:

- Which player averages the most kills per game?
- Which playlist gets played the most?
- Which map gets played the most?
- Which map do players get the most Killing Spree medals on?

In [99]:
%%sql

select player_gamertag, avg(player_total_kills) as avg_player_total_kills
from (
    select match_id , player_gamertag, player_total_kills
    from df_joined
    group by match_id , player_gamertag, player_total_kills
) as a
group by player_gamertag
order by avg_player_total_kills desc

player_gamertag,avg_player_total_kills
gimpinator14,109.0
I Johann117 I,96.0
BudgetLegendary,83.0
GsFurreal,75.0
Sexy is Back,73.0
killerguy789,68.0
THC GUILTYSPARK,67.0
HisLattice1,66.0
PrimePromethean,66.0
taurenmonk,64.0


Which player averages the most kills per game? Answer: gimpinator14 

In [100]:
%%sql

select player_gamertag, count(*) as player_total_games
from (
    select match_id , player_gamertag
    from df_joined
    group by match_id , player_gamertag
) as a
group by player_gamertag
order by player_total_games desc

player_gamertag,player_total_games
EcZachly,3291
ILLICIT 117,1769
Battle Drunk,1418
Ash All Mighty,1288
JakeWilson801,1269
HD DrAsTiC,1012
Amplafied,965
Stormbane321,837
Dope Boy,766
My Regret,633


Which playlist gets played the most? Answer: EcZachly

In [104]:
%%sql

select map_name, count(*) as map_total_games
from (
    select match_id ,  map_name
    from df_joined
    group by match_id ,  map_name
) as a
group by map_name
order by map_total_games desc

map_name,map_total_games
Breakout Arena,7032
Alpine,1358
Empire,1347
The Rig,1022
Truth,970
Plaza,949
Glacier,922
Regret,922
Coliseum,904
Eden,870


Which map gets played the most? Answer: Breakout Arena

In [109]:
%%sql

select map_name, sum(medals_cnt) as medals_sum
from (
    select match_id , player_gamertag ,  map_name, medal_name, sum(count) as medals_cnt
    from df_joined
    where medal_name = 'Killing Spree'
    group by match_id , player_gamertag ,  map_name, medal_name
) as a
group by map_name
order by medals_sum desc

map_name,medals_sum
Breakout Arena,6738.0
Alpine,5359.0
Glacier,3402.0
Empire,2233.0
Truth,2061.0
The Rig,2003.0
Plaza,1912.0
Coliseum,1910.0
Eden,1769.0
Regret,1761.0


Which map do players get the most Killing Spree medals on?  Answer: Breakout Arena

With the aggregated data set
Try different .sortWithinPartitions to see which has the smallest data size (hint: playlists and maps are both very low cardinality)

In [115]:
df_joined.explain(mode='cost')

== Optimized Logical Plan ==
Project [match_id#8172, player_gamertag#8173, previous_spartan_rank#8174, spartan_rank#8175, previous_total_xp#8176, total_xp#8177, previous_csr_tier#8178, previous_csr_designation#8179, previous_csr#8180, previous_csr_percent_to_next_tier#8181, previous_csr_rank#8182, current_csr_tier#8183, current_csr_designation#8184, current_csr#8185, current_csr_percent_to_next_tier#8186, current_csr_rank#8187, player_rank_on_team#8188, player_finished#8189, player_average_life#8190, player_total_kills#8191, player_total_headshots#8192, player_total_weapon_damage#8193, player_total_shots_landed#8194, player_total_melee_kills#8195, ... 36 more fields], Statistics(sizeInBytes=4.24E+30 B)
+- Join Inner, (mapid#542 = mapid#8209), rightHint=(strategy=broadcast), Statistics(sizeInBytes=4.31E+30 B)
   :- Project [match_id#8172, player_gamertag#8173, previous_spartan_rank#8174, spartan_rank#8175, previous_total_xp#8176, total_xp#8177, previous_csr_tier#8178, previous_csr_desig

In [118]:
mapid_sort_df = df_joined.repartition(16, col("mapid")).sortWithinPartitions(col("mapid"))
mapid_sort_df.createOrReplaceTempView('mapid_sort_df')
mapid_sort_df.explain(mode='cost')

== Optimized Logical Plan ==
Sort [mapid#8209 ASC NULLS FIRST], false, Statistics(sizeInBytes=4.24E+30 B)
+- RepartitionByExpression [mapid#8209], 16, Statistics(sizeInBytes=4.24E+30 B)
   +- Project [match_id#8172, player_gamertag#8173, previous_spartan_rank#8174, spartan_rank#8175, previous_total_xp#8176, total_xp#8177, previous_csr_tier#8178, previous_csr_designation#8179, previous_csr#8180, previous_csr_percent_to_next_tier#8181, previous_csr_rank#8182, current_csr_tier#8183, current_csr_designation#8184, current_csr#8185, current_csr_percent_to_next_tier#8186, current_csr_rank#8187, player_rank_on_team#8188, player_finished#8189, player_average_life#8190, player_total_kills#8191, player_total_headshots#8192, player_total_weapon_damage#8193, player_total_shots_landed#8194, player_total_melee_kills#8195, ... 36 more fields], Statistics(sizeInBytes=4.24E+30 B)
      +- Join Inner, (mapid#542 = mapid#8209), rightHint=(strategy=broadcast), Statistics(sizeInBytes=4.31E+30 B)
         :-

In [119]:
match_id_sort_df = df_joined.repartition(16, col("match_id")).sortWithinPartitions(col("match_id"))
match_id_sort_df.createOrReplaceTempView('match_id_sort_df')
match_id_sort_df.explain(mode='cost')

== Optimized Logical Plan ==
Sort [match_id#8172 ASC NULLS FIRST], false, Statistics(sizeInBytes=4.24E+30 B)
+- RepartitionByExpression [match_id#8172], 16, Statistics(sizeInBytes=4.24E+30 B)
   +- Project [match_id#8172, player_gamertag#8173, previous_spartan_rank#8174, spartan_rank#8175, previous_total_xp#8176, total_xp#8177, previous_csr_tier#8178, previous_csr_designation#8179, previous_csr#8180, previous_csr_percent_to_next_tier#8181, previous_csr_rank#8182, current_csr_tier#8183, current_csr_designation#8184, current_csr#8185, current_csr_percent_to_next_tier#8186, current_csr_rank#8187, player_rank_on_team#8188, player_finished#8189, player_average_life#8190, player_total_kills#8191, player_total_headshots#8192, player_total_weapon_damage#8193, player_total_shots_landed#8194, player_total_melee_kills#8195, ... 36 more fields], Statistics(sizeInBytes=4.24E+30 B)
      +- Join Inner, (mapid#542 = mapid#8209), rightHint=(strategy=broadcast), Statistics(sizeInBytes=4.31E+30 B)
     